In [ ]:
%pylab inline

In [ ]:
import katpoint
from IPython.core.display import display, HTML


This notebook can be used each week to plan the scheduling of observations on MeerKAT. 

It will generate an obs table, with times in SAST, that can be used to block off times in the Google calendar

See https://en.wikipedia.org/wiki/Sidereal_time for a discussion of siderial time

Some useful points to check your scheduling sanity:
* The RA of a target is equal to the LST when the target is at its highest elevation (when it tranits)... this is useful because if you are doing scheduling the best is to observe equally on either side of this time
* LST will change by roughy 4 mins per day ... so a target will transit 4 mins earlier each day
* MeerKAT LST matches SAST on roughly 30 Sep each year ... it is 6 hrs after SAST on about 30 Dec, 12 hrs difference in March and 6 hrs before in June 

You can also look at the following pyephem doc page for more ways to use the ant.ref_observer object. http://rhodesmill.org/pyephem/quick.html

Link to the MeerKAT observation calendar

https://calendar.google.com/calendar/embed?src=ska.ac.za_hd1ilp7tkoffoa9f2gh0j6l0q8%40group.calendar.google.com&ctz=Africa/Johannesburg

In [ ]:
ant = katpoint.Antenna('ant1, -30:42:39.8, 21:26:38.0, 1035.0') # the MeerKAT reference point
ant.ref_observer.horizon = '20:00:00' # horizon set to 20 degrees
SAST = matplotlib.dateutil.relativedelta.relativedelta(hours=2) # useful to convert to SAST (UTC + 2)

In [ ]:
# catalogue cell

cat = katpoint.Catalogue()
# MeerKAT main bp calibrators
cat.add(katpoint.Target('PKS 1934-63, radec bpcal, 19:39:25.03, -63:42:45.7'))
cat.add(katpoint.Target('J0137+3309 | 3C48,radec bpcal,01:37:41.29,+33:09:35.13'))
#cat.add(katpoint.Target('PKS 0408-65, radec, 4:08:20.38, -65:45:09.1'))
cat.add(katpoint.Target('J0408-6545 | PKS 0408-65, radec bpcal delaycal, 4:08:20.38, -65:45:09.1'))
#cat.add(katpoint.Target('proxima Cen, radec target, 14:29:42.94, -62:40:46.1'))
#cat.add(katpoint.Target('J1424-4913 | 1421-490,radec gaincal,14:24:32.08,-49:13:50.7'))
cat.add(katpoint.Target('GRB190114C, radec target, 03:38:01.17 , -26:56:46.73'))

In [ ]:
### start time cell

# select the timestamp to be used to generate the obs table 
# comment out one of the following two lines
#start_timestamp = katpoint.Timestamp() # use NOW as the start time for the obs table
start_timestamp = katpoint.Timestamp('2019-05-01 00:00') # use a specific time as the start time (in UTC ... )

In [ ]:
# pyephem version of the katpoint timestamp for the ref_observer object
start_ed = start_timestamp.to_ephem_date()

In [ ]:
# generate an html table for displaying
# only the first 16 characters [:16] are shown for the time string
# for readability

h = '<table>'
h += "<tr><td colspan='4'>Obs table for %s</td></tr>"%str(start_timestamp+7200)[:16]
h += "<tr><td>Target<td>Next Rise<td>Next Transit<td>Next Set</tr>"
for tar in cat.targets:
    h += '<tr><td> %s <td> %s <td> %s <td> %s </tr>'%(tar.name,
                                str((ant.ref_observer.next_rising(tar.body,start_ed).datetime()+SAST))[:16],
                                str((ant.ref_observer.next_transit(tar.body,start_ed).datetime()+SAST))[:16],
                                str((ant.ref_observer.next_setting(tar.body,start_ed).datetime()+SAST))[:16])
h += "<tr><td colspan='4'>These times are in SAST</td></tr>"
h += "</table>"
display(HTML(h))

Recomended scheduling steps:
0. Make copy of this notebook
1. Get the target coordinates
2. Get the required duration for the time on target ... call this (m)
3. Get the obs priority order from the Tuesday Comm meeting options are 
 * has to be observed this week
 * was missed last week
 * no priority
4. Edit the catalogue and start-time cells to generate the obs table
5. For the first target of interest, is the calendar free for m/2 either side of the transit time? If yes the book that time.
6. If the block time in the previous step is not available can you slide the block forward or backward between the rise and set time to find a free block? If you find a gap then book that time.
7. If you need to choose a day later in the week then remember that the times are about 4 mins earlier for each day after the obs table is generated ... this is usually not that significant (at most 30 mins) in a week 
8. Repeat until all the targets are scheduled
9. Fill in the gaps with filler observations like pointing or baseline cal

In [ ]:
tar=cat.targets[-1] # the last one
print ant.ref_observer.next_rising(tar.body,start_ed)
print ant.ref_observer.next_setting(tar.body,start_ed)

In [ ]:
print "Duration %6.2f Hours" %((ant.ref_observer.next_setting(tar.body,start_ed)\
                  - ant.ref_observer.next_rising(tar.body,start_ed))*24)

In [ ]:
print ant.ref_observer.sidereal_time()

In [ ]:
print ant.ref_observer.sidereal_time